In [22]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

from data import DataReader
from dilate.dilate import dilate_loss
import torch
import sys

sys.setrecursionlimit(100000)

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
dr = DataReader()
df = dr.train.query("id == 1")

In [4]:
windows = []
window_size = 144
for i in range(df.shape[0]):
    windows.append(df.iloc[i : i + window_size, -1].values)

In [5]:
loss = dilate_loss(alpha=0.5, gamma=0.001, device="cuda")
mse = torch.nn.MSELoss()

In [23]:
%%timeit
batch = 1

for i, _ in enumerate(windows):
    for j, _ in enumerate(windows):
        i = (
            torch.tensor(windows[i : i + batch])
            # .to("cuda")
            .view(batch, -1, 1)
        )
        j = (
            torch.tensor(windows[j : j + batch])
            # .to("cuda")
            .view(batch, -1, 1)
        )
        l = loss(i, j)
        # mse(i, j)
        # l.backward()
        break
    break

147 ms ± 9.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
from numba import cuda

In [25]:
print(cuda.gpus)

<Managed Device 0>, <Managed Device 1>, <Managed Device 2>, <Managed Device 3>, <Managed Device 4>, <Managed Device 5>, <Managed Device 6>, <Managed Device 7>
